# CSI 4142 Deliverable 2: Physical Design and Data Staging


Group 15:

Lixiong Wei: 300145970

Zhiyuan Lin: 300126813


This file is dealing for ETL process

This Jupyter file is used to combine all data from different provinces into one

We will joining the following attributes from these table:

covid19-download.csv: 

**totalcases, numdeaths**

For other 13 table from different provinces: 

**data » total_recoveries**

**data » total_vaccinations**       

**data » total_vaccinated**     

**data » total_boosters_1** 

In [1]:
# !pip install pandas
import pandas as pd

In [2]:
# # Read the csv file into a pandas dataframe
url = 'https://raw.githubusercontent.com/William-67/CSI4142_Group15/main/COVID19Tracker.ca%20Data%20-%20'
ab = pd.read_csv(url + 'AB.csv')
bc = pd.read_csv(url + 'BC.csv')
mb = pd.read_csv(url + 'MB.csv')
nb = pd.read_csv(url + 'NB.csv')
nl = pd.read_csv(url + 'NL.csv')
ns = pd.read_csv(url + 'NS.csv')
nt = pd.read_csv(url + 'NT.csv')
nu = pd.read_csv(url + 'NU.csv')
on = pd.read_csv(url + 'ON.csv')
pe = pd.read_csv(url + 'PE.csv')
sk = pd.read_csv(url + 'SK.csv')
qc = pd.read_csv(url + 'QC.csv')
yt = pd.read_csv(url + 'YT.csv')
df = pd.read_csv('https://raw.githubusercontent.com/William-67/CSI4142_Group15/main/covid19-download.csv')

In [3]:
#alternate method for reading file, need all files to be downloaded
# ab = pd.read_csv('COVID19Tracker.ca Data - AB.csv')
# bc = pd.read_csv('COVID19Tracker.ca Data - BC.csv')
# mb = pd.read_csv('COVID19Tracker.ca Data - MB.csv')
# nb = pd.read_csv('COVID19Tracker.ca Data - NB.csv')
# nl = pd.read_csv('COVID19Tracker.ca Data - NL.csv')
# ns = pd.read_csv('COVID19Tracker.ca Data - NS.csv')
# nt = pd.read_csv('COVID19Tracker.ca Data - NT.csv')
# nu = pd.read_csv('COVID19Tracker.ca Data - NU.csv')
# on = pd.read_csv('COVID19Tracker.ca Data - ON.csv')
# pe = pd.read_csv('COVID19Tracker.ca Data - PE.csv')
# sk = pd.read_csv('COVID19Tracker.ca Data - SK.csv')
# qc = pd.read_csv('COVID19Tracker.ca Data - QC.csv')
# yt = pd.read_csv('COVID19Tracker.ca Data - YT.csv')
# df = pd.read_csv('covid19-download.csv')

In [4]:
#test 
ab.dtypes

province                              object
last_updated                          object
data » date                           object
data » change_cases                    int64
data » change_fatalities               int64
data » change_tests                    int64
data » change_hospitalizations         int64
data » change_criticals                int64
data » change_recoveries               int64
data » change_vaccinations             int64
data » change_vaccinated               int64
data » change_boosters_1               int64
data » change_boosters_2               int64
data » change_vaccines_distributed     int64
data » total_cases                     int64
data » total_fatalities                int64
data » total_tests                     int64
data » total_hospitalizations          int64
data » total_criticals                 int64
data » total_recoveries                int64
data » total_vaccinations              int64
data » total_vaccinated                int64
data » tot

Only Select recovery data

**data » total_recoveries**

**data » total_vaccinations**       

**data » total_vaccinated**     

**data » total_boosters_1** 

In [5]:
origin_list = [ab,bc,mb,nb,nl,ns,nt,nu,on,pe,sk,qc,yt]
modify_list = []
for province in origin_list:
    temp = province[["data » date","province","data » total_recoveries","data » total_vaccinations","data » total_vaccinated","data » total_boosters_1"]]
    # temp = temp.dropna()
    temp = temp.rename({'data » date':'date','data » total_recoveries': 'recovery'},axis='columns')
    temp = temp.rename({'data » total_vaccinations':'partial'},axis='columns')
    temp = temp.rename({'data » total_vaccinated':'fully'},axis='columns')
    temp = temp.rename({'data » total_boosters_1':'booster'},axis='columns')
    temp['date'] = pd.to_datetime(temp['date'])

    temp['province'] = temp['province'][0]
    temp['province'] = temp['province'].str.upper()

    #Find the missing date and fill them with closest date data
    missing_dates = pd.date_range(start=temp['date'].min(), end='2023-12-31').difference(temp['date'])
    filled_rows = []

    for date in missing_dates:
        closest_date = temp['date'].iloc[(temp['date'] - date).abs().argsort()[0]]
        closest_row = temp[temp['date'] == closest_date].iloc[0]
        filled_rows.append({'date': date, 'province': closest_row['province'], 
                            'recovery': closest_row['recovery'], 'partial': closest_row['partial'],
                            'fully': closest_row['fully'], 'booster': closest_row['booster']})
    filled_table = pd.DataFrame(filled_rows)
    temp = pd.concat([temp,filled_table],ignore_index=True)
    # print(temp.head(5))
    # print(missing_dates)
    # print(temp.dtypes)
    modify_list.append(temp)
    
# print(modify_list[4])

In [6]:
#merge all data into one
merge_data = pd.concat(modify_list)
merge_data.sort_values(by='date', inplace=True)
merge_data.reset_index(drop=True, inplace=True)
print(merge_data.head(20))

         date province  recovery  partial  fully  booster
0  2020-01-25       AB         0        0      0        0
1  2020-01-25       NT         0        0      0        0
2  2020-01-25       ON         0        0      0        0
3  2020-01-25       NS         0        0      0        0
4  2020-01-25       PE         0        0      0        0
5  2020-01-25       NL         0        0      0        0
6  2020-01-25       SK         0        0      0        0
7  2020-01-25       NB         0        0      0        0
8  2020-01-25       QC         0        0      0        0
9  2020-01-25       MB         0        0      0        0
10 2020-01-25       YT         0        0      0        0
11 2020-01-25       BC         0        0      0        0
12 2020-01-25       NU         0        0      0        0
13 2020-01-26       NU         0        0      0        0
14 2020-01-26       MB         0        0      0        0
15 2020-01-26       PE         0        0      0        0
16 2020-01-26 

covid19-download.csv: 

**totalcases, numdeaths**

In [7]:
cases = df[['date','prname','totalcases','numdeaths']]
cases['date'] = pd.to_datetime(cases['date'])

C:\Users\weili\AppData\Local\Temp\ipykernel_11256\3832118507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['date'] = pd.to_datetime(cases['date'])


**Write a new column so that inner join can be perform smoothly**

In [8]:
province_mapping = {
    'Ontario': 'ON',
    'Quebec': 'QC',
    'Nova Scotia': 'NS',
    'New Brunswick': 'NB',
    'Manitoba': 'MB',
    'British Columbia': 'BC',
    'Prince Edward Island': 'PE',
    'Saskatchewan': 'SK',
    'Alberta': 'AB',
    'Newfoundland and Labrador': 'NL',
    'Northwest Territories': 'NT',
    'Yukon': 'YT',
    'Nunavut': 'NU'
}
cases['province'] = cases['prname'].map(province_mapping)
cases = cases[['date','prname','province','totalcases','numdeaths']]


C:\Users\weili\AppData\Local\Temp\ipykernel_11256\3542065097.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases['province'] = cases['prname'].map(province_mapping)


In [9]:
cases.head(20)

,date,prname,province,totalcases,numdeaths
0,2020-02-01,British Columbia,BC,1,0
1,2020-02-01,Alberta,AB,0,0
2,2020-02-01,Saskatchewan,SK,0,0
3,2020-02-01,Manitoba,MB,0,0
4,2020-02-01,Ontario,ON,3,0
5,2020-02-01,Quebec,QC,0,0
6,2020-02-01,Newfoundland and Labrador,NL,0,0
7,2020-02-01,New Brunswick,NB,0,0
8,2020-02-01,Nova Scotia,NS,0,0
9,2020-02-01,Prince Edward Island,PE,0,0


**Merge dataset started**


In [10]:
import copy

complete_table = copy.copy(cases)

# # Merge the tables based on date and province using merge_asof
# merged_df = pd.merge_asof(complete_table.sort_values('date'), merge_data.sort_values('date'), on='date', by='province', direction='nearest')

# # Print the merged dataframe
# print(merged_df)
# merged_df.to_csv("testing_table.csv", index=False) 


complete_table = pd.merge(complete_table, merge_data,how="inner", on=["province","date"])



In [11]:
#Selece the date from 2022 January to 2023 December
filtered_data = complete_table[(complete_table['date'] >= '2021-01-01') & (complete_table['date'] <= '2023-12-31')]
complete_table = filtered_data
complete_table.reset_index(drop=True, inplace=True)
complete_table

,date,prname,province,totalcases,numdeaths,recovery,partial,fully,booster
0,2021-01-02,British Columbia,BC,53162,901,42129,17510,0,0
1,2021-01-02,Alberta,AB,104633,1267,84827,14244,0,0
2,2021-01-02,Saskatchewan,SK,15844,158,12975,3722,0,0
3,2021-01-02,Manitoba,MB,25026,678,19789,3540,2,0
4,2021-01-02,Ontario,ON,193960,4905,160526,33191,0,0
...,...,...,...,...,...,...,...,...,...
2036,2023-12-30,Nova Scotia,NS,150811,961,43018,2235685,842600,503257
2037,2023-12-30,Prince Edward Island,PE,58228,114,41242,391227,144466,83158
2038,2023-12-30,Yukon,YT,4989,32,4489,92743,35380,20672
2039,2023-12-30,Northwest Territories,NT,11511,22,11327,102863,39663,20278


**Add surrogate keys**

In [12]:
complete_table['Surrogate Keys'] = range(1,len(complete_table)+1)
surrogate_keys_column = complete_table.pop('Surrogate Keys')
complete_table.insert(0, 'Surrogate Keys', surrogate_keys_column)
complete_table

,Surrogate Keys,date,prname,province,totalcases,numdeaths,recovery,partial,fully,booster
0,1,2021-01-02,British Columbia,BC,53162,901,42129,17510,0,0
1,2,2021-01-02,Alberta,AB,104633,1267,84827,14244,0,0
2,3,2021-01-02,Saskatchewan,SK,15844,158,12975,3722,0,0
3,4,2021-01-02,Manitoba,MB,25026,678,19789,3540,2,0
4,5,2021-01-02,Ontario,ON,193960,4905,160526,33191,0,0
...,...,...,...,...,...,...,...,...,...,...
2036,2037,2023-12-30,Nova Scotia,NS,150811,961,43018,2235685,842600,503257
2037,2038,2023-12-30,Prince Edward Island,PE,58228,114,41242,391227,144466,83158
2038,2039,2023-12-30,Yukon,YT,4989,32,4489,92743,35380,20672
2039,2040,2023-12-30,Northwest Territories,NT,11511,22,11327,102863,39663,20278


Load the data into new CSV file

In [13]:
complete_table.to_csv("final_result.csv", index=False) 

Create a SQL file for create a new table in PostgreSQL

In [14]:
import csv

# Define your CSV file path
csv_file_path = 'final_result.csv'

create_table_statement = """
CREATE TABLE IF NOT EXISTS coviddata (
    Surrogate_Keys SERIAL PRIMARY KEY,
    date DATE,
    prname VARCHAR(50),
    province VARCHAR(50),
    totalcases INTEGER,
    numdeaths INTEGER,
    recovery INTEGER,
    partial INTEGER,
    fully INTEGER,
    booster INTEGER
);
"""

# Read data from the CSV file and store it in a list of tuples
data_list = []
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        # Assuming each row in the CSV corresponds to one tuple of data
        data_list.append(tuple(row))

# Generate SQL insert statements
insert_statements = []
count = 0
for row in data_list:
    if count == 0:
        count = 1
        continue
    values = ', '.join([f"'{str(value)}'" if isinstance(value, str) else str(value) for value in row])
    insert_statements.append(f"INSERT INTO coviddata VALUES ({values});")

# Write SQL statements to a file
with open('data_loading.sql', 'w') as f:
    f.write(create_table_statement + '\n')
    f.write('\n'.join(insert_statements))